In [2]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForSequenceClassification ,AutoConfig, Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
from datasets import Features, Value
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns
import langchain
from bertopic import BERTopic
from sentence_transformers  import SentenceTransformer,CrossEncoder
from sentence_transformers.util import semantic_search
cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)

def pd_full_screen():
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    
def pd_normal_screen():
    pd.set_option('display.max_rows', 10)
    pd.set_option('display.max_columns', 5)



No ROCm runtime is found, using ROCM_HOME='/usr'


[2024-07-14 18:28:22,435] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


0it [00:00, ?it/s]

In [ ]:
df = pd.read_excel('alldata_cleaned.xlsx',dtype={'query':str,'element':str},index_col=0)
c = df['element'].str.contains('<form')
df = df[~c].drop_duplicates(['query']).reset_index(drop=True)

c2 = df['element'].str.contains('<a')

def link(x):
    a = BeautifulSoup(str(x['element']),'html.parser').find('a')
    return {'query':a.text.lower().strip(),'element':a}
d = df[c2].apply(link,axis=1)
d= pd.DataFrame(d.tolist())
d

In [ ]:
d = pd.read_excel('train_dataset.xlsx',index_col=0)
d

In [ ]:
queries = d['label'].unique().tolist()
for x in [0,1,2,3,4,5,6,7,8,9,10,11,12,13,'0','1']:
    queries.remove(x)
queries

In [ ]:
pd_normal_screen()
#d = drop_duplicates(['query']).reset_index(drop=True)
d =d.dropna(subset= ['query']).reset_index(drop=True)
d

In [7]:
d = pd.read_excel('true_form.xlsx',index_col=0)

queries = d['label'].unique()
d

,query,element,label
0,Please enter your personal data below Login \n...,"<form action=""/Vacancies/2224/Application/New/...",application form
1,Werkstudent mwd Forschung und Entwicklung Bew...,"<form action=""/Vacancies/14633/Application/New...",application form
2,Start Application First nameLast nameEmailAppl...,"<form>First name<input label=""First name"" name...",application form
3,Apply 1Personal Details\n \n W...,"<form accept-charset=""UTF-8"" action=""/postings...",application form
4,Career Opportunities Field Supervisor mwd 5296...,"<form accept-charset=""UTF-8"" action=""/career?_...",application form
5,If you have previously started an application ...,"<form action=""https://nottinghamshire.tal.net/...",next
6,Social Sign up\n \n\n\n sign up\n ...,"<form aria-label=""sign up"" novalidate="""" role=...",register form
7,Login Email\n \n\n\...,"<form action=""/cop-data/bd104c9e-a55a-4e72-aa1...",login form
8,Career Opportunities Sign In JavaScript is tur...,"<form accept-charset=""UTF-8"" action=""/career?_...",login form
9,Lets stay connected Register to receive regula...,"<form action=""https://career5.successfactors.e...",newsletter


In [5]:
model_name_bi_encoder = 'mixedbread-ai/mxbai-embed-large-v1'#'sentence-transformers/all-mpnet-base-v2'
model_name_bi_encoder = 'sentence-transformers/stsb-xlm-r-multilingual'
model_name_bi_encoder = 'intfloat/multilingual-e5-large-instruct'



bi_encoder = SentenceTransformer(model_name_bi_encoder,cache_folder=cache_dir).to('cuda')
bi_encoder

SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [1]:
#queries=d['label'].unique() #['sign in','log in','sign up','register','apply now','next','continue','cookie setting','accept cookie','search','newsletter','email','cv','cover letter','forget password','password']
queries_embed = bi_encoder.encode(queries,normalize_embeddings=False)
docs_embed = bi_encoder.encode(d['query'],normalize_embeddings=False)#,convert_to_tensor=True)
sim = bi_encoder.similarity(docs_embed,queries_embed)
sim.shape

NameError: name 'bi_encoder' is not defined

In [ ]:
"""passages = d['query']
hits = semantic_search(queries_embed,docs_embed,top_k=3)
for q,qh in zip(queries,hits):
    for h in qh:
        print(q,passages[h['corpus_id']])
        pass
        
    break


hits = semantic_search(docs_embed,queries_embed,top_k=3)
for p,qh in zip(passages,hits):
    for h in qh:
        print(p,queries[h['corpus_id']])
        
    break"""

In [9]:
pd_full_screen()
d['bi_label'] = pd.DataFrame(sim.argmax(1))
d['bi_value'] = pd.DataFrame(sim.max(1)[0])

d['bi_label'] = d['bi_label'].map(lambda x:queries[x])
c = d['bi_value']>0.5
d[c].sort_values('bi_value',ascending=False)

,query,element,label,bi_label,bi_value
14,Search,"<form name=""searchBox""><input autocomplete=""of...",search form,search form,0.942613
16,Keywords\n\nLocation\n\n\nSearch Jobs,"<form action=""/en/search-jobs"" data-ambient-se...",search form,search form,0.865951
5,If you have previously started an application ...,"<form action=""https://nottinghamshire.tal.net/...",next,application form,0.843167
18,Current Page Search jobs\nSearch jobs\n\nKeywo...,"<form action=""/en/search-jobs"" data-ambient-se...",search form,search form,0.842358
9,Lets stay connected Register to receive regula...,"<form action=""https://career5.successfactors.e...",newsletter,newsletter,0.819612
13,OF ENERGY Search by Job Number Title or Keywor...,"<form action="""" au-target-id=""137"" data-ph-id=...",search form,search form,0.819408
3,Apply 1Personal Details\n \n W...,"<form accept-charset=""UTF-8"" action=""/postings...",application form,application form,0.816528
7,Login Email\n \n\n\...,"<form action=""/cop-data/bd104c9e-a55a-4e72-aa1...",login form,login form,0.811397
2,Start Application First nameLast nameEmailAppl...,"<form>First name<input label=""First name"" name...",application form,application form,0.809869
0,Please enter your personal data below Login \n...,"<form action=""/Vacancies/2224/Application/New/...",application form,login form,0.799904


: 

In [55]:
c = d['bi_label']==d['label']
x = d[c].reset_index(drop=True)
x
#pd_normal_screen()


,query,element,...,bi_label,bi_value
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",...,email,0.935702
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",...,cookie,0.918792
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",...,cookie,0.906384
3,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",...,email,0.894877
4,search job by location,"<input aria-haspopup=""listbox"" aria-labelledby...",...,search,0.907292
...,...,...,...,...,...
833,privacy policy,"<a class=""privacy-policy"" data-turbo=""false"" h...",...,privacy,0.963721
834,og in,"<button class=""careersite-button careersite-bu...",...,login,0.893679
835,upload resume,"<a href=""javascript: void(0);"" title=""upload r...",...,cv,0.924190
836,upload fileprivacy policy,"<a href=""https://vertriebskarriere.hansemerkur...",...,privacy,0.908539


In [65]:
x = x.drop_duplicates(subset=['element','query']).reset_index(drop=True)
pd_full_screen()
x

,query,element,label,action,utterance,bi_label,bi_value
0,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",email,NaN,NaN,email,0.935702
1,Accept additional cookies,"<input id=""accept_all_cookies_button"" messaged...",cookie,NaN,NaN,cookie,0.918792
2,Reject additional cookies,"<input id=""accept_essential_cookies_button"" me...",cookie,NaN,NaN,cookie,0.906384
3,Recipient’s Email address,"<input aria-describedby="""" aria-label=""Please ...",email,NaN,NaN,email,0.894877
4,search job by location,"<input aria-haspopup=""listbox"" aria-labelledby...",search,NaN,NaN,search,0.907292
5,latitude,"<input aria-label=""latitude"" name=""latitude"" v...",distance,NaN,NaN,distance,0.914477
6,longitude,"<input aria-label=""longitude"" name=""longitude""...",distance,NaN,NaN,distance,0.939987
7,country,"<input aria-label=""country"" name=""country"" val...",country,NaN,NaN,country,1.000000
8,city,"<input aria-label=""city"" name=""city"" value=""""/>",city,NaN,NaN,city,1.000000
9,county,"<input aria-label=""county"" name=""county"" value...",city,NaN,NaN,city,0.911302


In [ ]:
#model_name = 'cross-encoder/stsb-roberta-large'#'cross-encoder/msmarco-MiniLM-L12-en-de-v1'#'cross-encoder/nli-distilroberta-base'
#model_name = 'sentence-transformers/stsb-xlm-r-multilingual'#'intfloat/multilingual-e5-large-instruct'
#model = AutoModelForSequenceClassification.from_pretrained(model_name,cache_dir=cache_dir)
#config = AutoConfig.from_pretrained(model_name,cache_dir=cache_dir)
model = CrossEncoder(model_name,cache_dir=cache_dir,device='cuda')
#model
model

In [ ]:
df = {}
for query in queries:
    inp = [[query,q['query']] for i,q in d.iterrows()]
    preds = model.predict(inp)
    df[query]=preds


In [ ]:
df = pd.DataFrame.from_dict(df)
#df

In [ ]:
pd_full_screen()
d['label']= df.idxmax(1)
d['label_value'] = df.max(axis=1)


In [ ]:
c = d['label_value']>0.75
d[c].sort_values('label_value',ascending=False)

In [ ]:
d['res']=res#[:,1]
pd_full_screen()
d.sort_values('res',ascending=False)
